- rename hony_p03_t002.csv to hony_2015.csv
- rename hony_p04_t003.csv to hony_2016.csv
- rename hony_p04_t020.csv to hony_2017.csv

In [193]:
import numpy as np
import pandas as pd
import os
import re
import csv

def honey_production(file_name):
    # get path
    path = os.getcwd()
    
    # open the file
    with open(path+'/'+file_name) as input_honey:
        honeyread = csv.reader(input_honey, delimiter = ',')
        honey  = []
        header = []
        for line in honeyread:
            if line[1] == 'd' and line[3] != '':
                honey.append(line[2:])
            if line[1] == 'h': #and line[3] != '': 
                header.append(line[2:])
            
    # get the headers, and indexes
    i = 0
    count = 0
    index = []
    title = []
    for line in header:
        if 'State' in line:
            count += 1 
            if line[2] == '':
                index_i = [x for x in header[i-1]]
                # index_i = [str(year) + x for x in header[i-1]]
                index.append(index_i[1:])
                title_i = [a+' '+b for a,b in zip(header[i+1],header[i+2])]
                title_clean = [re.sub(r'\d+\W','',c) for c in title_i]
                title.append(title_clean[1:])
            if line[2] != '':
                title_j = [a + ' ' + b + ' ' + e for a,b,e in zip(header[i-1], header[i], header[i+1] )]
                line_clean =  [re.sub(r'\d+\W','',c) for c in title_j]
                title.append(line_clean[1:])
        i+=1

    
    # put colonies data into df
    honey_df = pd.DataFrame(honey)

    honey_df = honey_df[honey_df[0] != 'United States 6/ 7/']
    honey_df = honey_df.drop(honey_df.columns[3:7], axis=1)
    
    # remove the unwanted characters & numbers from the indexes 
    honey_df[0] = honey_df[0].str.replace('\d+\W','')
    
    # create sepearte dataframes
    df_list=[]
    title[0] = title[0][0:2]
    
    for i in range(count):
        columns = [' '.join(col.split()).lower() for col in title[i]] # strip white spaces and make lower
        group_df = honey_df.iloc[:,1:len(columns)+1].groupby(honey_df.iloc[:,0]).nth(i)
        df = pd.DataFrame(group_df.values, columns = columns, index = group_df.index)
        
        df.index.name = None # remove unwanted index name
        
        # deal with the missing values 
        df = df.replace('-',0) # replace '-' with 0 (according to documentation)
        df = df.replace('(Z)', 0.5) # replace '(Z)' with 0.5 (according to documentation)
        df = df.apply(pd.to_numeric) # change the object type to int 

        # put the df's into a list, numbers of dfs should equal to 'count'
        df_list.append(df)
        
   
        
    # return(df_list, title, index,count,i) -> checked the number of dfs are correct
    return(df_list, title, index)
        
honey_production('hony_2017.csv')

([                honey producing colonies  yield per colony
  Alabama                                7                45
  Arizona                               22                40
  Arkansas                              29                77
  California                           335                41
  Colorado                              33                43
  Florida                              205                43
  Georgia                               99                32
  Hawaii                                12               131
  Idaho                                 95                44
  Illinois                              11                46
  Indiana                                7                48
  Iowa                                  35                58
  Kansas                                 7                79
  Kentucky                               5                38
  Louisiana                             43                81
  Maine                 

In [191]:
# 2017 data
df_17, title, index= honey_production('hony_2017.csv')
honey_prod_17 = pd.DataFrame(df_17[0])

# 2016 data
df_16, title, index= honey_production('hony_2016.csv')
honey_prod_16 = pd.DataFrame(df_16[0])

# 2015 data
df_15, title, index= honey_production('hony_2015.csv')
honey_prod_15 = pd.DataFrame(df_15[0])

# concat
honey_be = {'2015':honey_prod_15, 
            '2016':honey_prod_16,
            '2017':honey_prod_17
           }
honey_df = pd.concat(honey_be, sort=False)
honey_df

honey producing colonies  yield per colony
2015 Alabama                                7                47
     Arizona                               26                49
     Arkansas                              24                72
     California                           275                30
     Colorado                              29                51
     Florida                              220                54
     Georgia                               69                40
     Hawaii                                14               102
     Idaho                                 89                32
     Illinois                               8                51
     Indiana                                6                53
     Iowa                                  36                50
     Kansas                                 8                36
     Kentucky                               5                46
     Louisiana                             44                99
     Maine                                 10                47
     Michigan                              90                58
     Minnesota                            122                68
     Mississippi                           15                83
     Missouri                              10                52
     Montana                              146                83
     Nebraska                              57                48
     New Jersey                            12                27
     New York                              58                62
     North Carolina                        12                45
     North Dakota                         490                74
     Ohio                                  17                50
     Oregon                                71                38
     Other States                          30                39
     Pennsylvania                          17                53
...                                       ...               ...
2017 Iowa                                  35                58
     Kansas                                 7                79
     Kentucky                               5                38
     Louisiana                             43                81
     Maine                                 12                33
     Michigan                              87                45
     Minnesota                            126                62
     Mississippi                           13                80
     Missouri                               8                65
     Montana                              145                72
     Nebraska                              42                63
     New Jersey                            13                28
     New York                              57                56
     North Carolina                        11                41
     North Dakota                         455                74
     Ohio                                  15                73
     Oregon                                78                40
     Other States                          34                43
     Pennsylvania                          16                46
     South Carolina                        16                34
     South Dakota                         255                57
     Tennessee                              7                41
     Texas                                120                66
     Utah                                  27                31
     Vermont                                7                45
     Virginia                               5                35
     Washington                            77                45
     West Virginia                          6                40
     Wisconsin                             53                56
     Wyoming                               39                53

[123 rows x 2 columns]